# Indexing notebook

### First we need to move to the top-level directory.

In [1]:
cd ../..

/Users/leo/SourceTreeGit/FlexNeuART.refact2021


### Lucene indexer options

In [2]:
!scripts/index/create_lucene_index.sh

Using collection root: collections
Specify collection sub-directory, e.g., msmarco_pass (1st arg)
Usage: <collection> [additional options]
Additional options:
-h print help
-exact_match create index for exact match
-index_field indexing field name (default text)
-input_subdir input data sub-directory (default input_data)
-index_subdir index subdirectory (default lucene_index)


By default Lucene using the context of the field `text` to create the full-text index, which is stored in the sub-directory `lucene_index`, but it is possible to create an index for an exact match, use a content of a different field, or store the inde

### Lucene index

In [ ]:
!scripts/index/create_lucene_index.sh wikipedia_dpr_nq_sample

### Forward indices

A forward index allows one to retrieve an original or parsed field content using a document identifier. The forward index can be stored using two backends (storage "engines"). There are four types of the field, which include:
1. two parsed textual field formats: 
    - Parsed text without positional information (bag-of-words): **parsedBOW**   
    - Parsed text with positional information: **parsedText**
2. original/unparsed/raw text: **textRaw**
3. binary (can be anything): **binary**

The options are printed by the indexing script:

In [ ]:
!scripts/index/create_fwd_index.sh 

Here we create indices sequentially, but they can also be created **in parallel** (independently for each field):

In [ ]:
!scripts/index/create_fwd_index.sh wikipedia_dpr_nq_sample mapdb \
'dense:binary text:parsedText text_unlemm:parsedText title:parsedBOW text_bert_tok:parsedText text_raw:textRaw'